In [18]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Conv1D, GlobalMaxPooling1D, Dense, Dropout
from sklearn.model_selection import train_test_split
import pandas as pd

# Load and preprocess the dataset
train_data = pd.read_csv('train.csv')
test_data = pd.read_csv('test.csv')

max_len = 50  # Adjust as needed
embedding_dim = 50  # Adjust as needed

# Tokenize and pad sequences
tokenizer = Tokenizer()
tokenizer.fit_on_texts(train_data['text'])
X_train = pad_sequences(tokenizer.texts_to_sequences(train_data['text']), maxlen=max_len, padding='post')
X_test = pad_sequences(tokenizer.texts_to_sequences(test_data['text']), maxlen=max_len, padding='post')

# Labels
y_train = train_data['label']
y_test = test_data['label']


In [20]:
import tensorflow as tf
from transformers import TFAutoModelForSequenceClassification

# Load the pre-trained model for sequence classification
model = TFAutoModelForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=1)

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


/Users/mahsaraeisinezhad/code/apps/brainPOP/venv/lib/python3.11/site-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [22]:
# Train the model
model.fit(X_train, y_train, epochs=3, batch_size=32)


Epoch 1/3
268/268 [==============================] - 634s 2s/step - loss: 4.4988 - accuracy: 0.7025
Epoch 2/3
268/268 [==============================] - 638s 2s/step - loss: 4.5083 - accuracy: 0.7044
Epoch 3/3
268/268 [==============================] - 604s 2s/step - loss: 4.5083 - accuracy: 0.7044


In [27]:
class Grader:
    def __init__(self, model, tokenizer):
        self.model = model
        self.tokenizer = tokenizer

    def evaluate(self, text):
        # Preprocess the input text
        encoded_text = self.tokenizer.texts_to_sequences([text])
        encoded_text = pad_sequences(encoded_text, maxlen=max_len, padding='post')

        # Make a prediction
        prediction = self.model.predict(encoded_text)[0].squeeze()

        return prediction.item()

# Create an instance of the Grader class with the trained model
mygrader = Grader(model, tokenizer)

# Example usage
output = mygrader.evaluate("He don’t come here no more.")
print(output)


1/1 [==============================] - 1s 939ms/step
4.164810657501221


In [ ]:
# Evaluate the performance on the test set
test_predictions = model.predict(test_texts)
# Compute precision and recall using test_predictions and test_labels
# You may need to threshold the predictions based on your specific needs
